In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random

from CV101.models import Sequential, load_model
from CV101.optimizers import SGD, Momentum, RMSprop, Adam
from CV101.metrics import accuracy, mae, precision, recall
from CV101.layers import Dense, Conv2D, MaxPool2D, Dropout, BatchNormalization, Flatten
from CV101.losses import MeanSquaredError, CategoricalCrossEntropy, BinaryCrossEntropy
from CV101.activations import ReLU, LeakyReLU, ELU, Sigmoid, Tanh, Linear
from CV101.initializers import HeInitializer, XavierInitializer
from CV101.regularizers import regularizers

# 1. Titanic

## 1.1 Preprocessing

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer


In [ ]:
while os.getcwd().split("\\")[-1] != "ComputerVisionSoc":
    os.chdir("..")

os.getcwd()

In [ ]:
train = pd.read_csv("datasets\\titanic\\train.csv", index_col="PassengerId")
train.head()

In [ ]:
train.info()

In [ ]:
train.drop(columns=["Ticket", "Cabin", "Name"], inplace=True)

In [ ]:
train.info()

In [ ]:
train.isna().sum()

In [ ]:
train.Embarked.value_counts()

In [ ]:
train.info()

In [ ]:
cleaned = train.dropna()
cleaned.isna().sum()


In [ ]:
encoder = OneHotEncoder(sparse_output=False)
encoded = np.array(encoder.fit_transform(cleaned[["Sex", "Embarked"]]))
encoded_name = encoder.get_feature_names_out(["Sex", "Embarked"])

encoded_df = pd.DataFrame(encoded, columns=encoded_name, index=cleaned.index)
encoded_df

In [ ]:
training = pd.concat([cleaned, encoded_df], axis=1)
training.drop(columns=["Sex", "Embarked"], inplace=True)
training.info()

In [ ]:
training.isna().sum()

In [ ]:
X_train = training.drop(columns=["Survived"])
y_train = training[["Survived"]].copy()

X_train.shape, y_train.shape

In [ ]:
X, y, X_val, y_val = X_train[:600], y_train[:600], X_train[600:], y_train[600:]

## 1.2 Create Model

In [ ]:
model1 = Sequential()
model1.add(Dense(64, activation=ReLU(), initializer=HeInitializer(mode="Normal")))
model1.add(Dense(32, activation=ReLU(), initializer=HeInitializer(mode="Normal")))
model1.add(Dense(10, activation=ReLU(), initializer=HeInitializer(mode="Normal")))
model1.add(Dense(1, activation=Sigmoid(), initializer=XavierInitializer(mode="Normal")))


model1.compile(loss=BinaryCrossEntropy(), optimizer=Adam(learning_rate=1e-4), metrics=[accuracy])

In [ ]:
y_pred = model1.predict(X)
y_pred[:10]

In [ ]:
accuracy(y, y_pred)

In [ ]:
model1.fit(X, y, validation_data = [X_val, y_val], epoch=500)

In [ ]:
model1.predict(X_val)

In [ ]:
accuracy(y_val, model1.predict(X_val))

## 1.3 Testing Model

In [ ]:
test = pd.read_csv("datasets\\titanic\\test.csv", index_col="PassengerId")
test.info()

In [ ]:
test.drop(columns=["Cabin", "Name", "Ticket"], inplace=True)

In [ ]:
test.isna().sum()

In [ ]:
imputer = SimpleImputer(strategy="mean")
test[["Age"]] = imputer.fit_transform(test[["Age"]])
test[["Fare"]] = imputer.fit_transform(test[["Fare"]])

test.isna().sum()

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
encoded = np.array(encoder.fit_transform(test[["Sex", "Embarked"]]))
encoded_name = encoder.get_feature_names_out(["Sex", "Embarked"])

encoded_df = pd.DataFrame(encoded, columns=encoded_name, index=test.index)
encoded_df

In [ ]:
testing = pd.concat([test, encoded_df], axis=1)
testing.drop(columns=["Sex", "Embarked"], inplace=True)
testing.info()

In [ ]:
testing.shape

In [ ]:
y_pred = model1.predict(testing)
y_pred.shape

In [ ]:
y_true = pd.read_csv("datasets\\titanic\\gender_submission.csv", index_col="PassengerId")
y_true

In [ ]:
y_true.shape

In [ ]:
print("Accuracy: ", accuracy(y_true, y_pred))

In [ ]:
model1.save_model("model1")

In [ ]:
model_test = load_model("model1")

In [ ]:
y_pred = model_test.predict(testing)
print("Accuracy: ", accuracy(y_true, y_pred))

In [ ]:
y_pred

# 2. MNIST

## 2.1 Preprocessing

In [ ]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test)= tf.keras.datasets.mnist.load_data()

In [ ]:
X_train, X_test, Y_train, Y_test = np.expand_dims(x_train, axis=3), np.expand_dims(x_test, axis=3), np.expand_dims(y_train, axis=1), np.expand_dims(y_test, axis=1)

# X_train, Y_train, X_test, Y_test = X_train[:1000], Y_train[:1000], X_test[:100], Y_test[:100]
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [ ]:
X_train, X_test = X_train/255, X_test/255

In [ ]:
plt.imshow(X_train[0])

In [ ]:
plt.hist(Y_train)

In [ ]:
new_Y_train = np.zeros((60000, 10))
# print(len(Y_train))
for i in range(len(Y_train)):
    # print(i, Y_train[i][0])
    new_Y_train[i, Y_train[i][0]] = 1

new_Y_test = np.zeros((10000, 10))
# print(len(Y_train))
for i in range(len(Y_test)):
    # print(i, Y_train[i][0])
    new_Y_test[i, Y_test[i][0]] = 1


new_Y_train.shape, new_Y_test.shape

In [ ]:
Y_train, Y_test = new_Y_train, new_Y_test

## 2.2 Create model

In [ ]:
model2 = Sequential()

model2.add(Conv2D(64, 3, activation=ReLU(), initializer=XavierInitializer(mode="normal")))
model2.add(MaxPool2D((2, 2)))

model2.add(Conv2D(32, 3, activation=ReLU(), initializer=HeInitializer(mode="normal")))
model2.add(MaxPool2D((2, 2)))

model2.add(Flatten())
# model1.add(BatchNormalization())

model2.add(Dense(64, ReLU(), initializer=XavierInitializer(mode="normal")))
model2.add(Dropout(0.5))
model2.add(Dense(32, ReLU(), initializer=XavierInitializer(mode="normal")))
model2.add(Dropout(0.5))
model2.add(Dense(10, activation=Linear(), initializer=XavierInitializer(mode="uniform")))

model2.compile(loss=CategoricalCrossEntropy(), optimizer=Adam(learning_rate=1e-4), metrics=[accuracy, precision, recall])

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
y_pred[random.choice(range(0, len(y_pred)))]

In [ ]:
accuracy(Y_test, y_pred)

In [ ]:
[np.mean(np.abs(model2.layers[i].W)) for i in range(len(model2.layers)) if model2.layers[i].trainable]

In [ ]:
plt.hist(y_pred.reshape(-1), 100)
plt.show()

In [ ]:
model2.fit(X_train, Y_train, epoch=10, batch_size=128, validation_data=(X_test, Y_test))

## 2.3 Testing

In [ ]:
random_num = random.choice(range(len(X_test)))
X_rand = X_test[random_num]
plt.imshow(X_rand)
plt.show()

X_rand = np.expand_dims(X_rand, axis=0)
# print(X_rand.shape)
print(f"Prediction: {np.argmax(model2.predict(X_rand))}")

In [ ]:
Y_pred = model2.predict(X_test)

In [ ]:
Y_pred

In [ ]:
accuracy(Y_pred, Y_test)

In [ ]:
model2.save_model("model2")